In [1]:
# standard python imports
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import copy

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [8]:
data = np.load('exp_io_data\CoupledPendula_mean_in7_out7_Tmax2.0_data.npz')
for key, val in data.items():
    print(key)
    exec(key +'=val')

xlist
exp_out_list


In [9]:
x=xlist
y=exp_out_list
#print(y)

In [10]:
print(x.shape)
print(y.shape)

(1000, 14)
(2, 60, 1000, 7, 2)


In [11]:
y1=y[0:1,29:30,:,:,0:1]
print(y.shape)

#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
y1_np=torch.from_numpy(y1)
y1=torch.transpose(y1_np,1,2)
print(y1.shape)
y2=y[0:1,7:8,:,:,0:1]
#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
y2_np=torch.from_numpy(y2)
y2=torch.transpose(y2_np,1,2)
print(y2.shape)
#print(y1.shape)

(2, 60, 1000, 7, 2)
torch.Size([1, 1000, 1, 7, 1])
torch.Size([1, 1000, 1, 7, 1])


In [12]:
y_new=torch.cat((y1,y2),2)
print(y_new.shape)
y_new=y_new.squeeze(0).squeeze(3)
print(y_new.shape)

torch.Size([1, 1000, 2, 7, 1])
torch.Size([1000, 2, 7])


In [13]:
y3=y_new[:,1:2,3:4]
print(y3.shape)

torch.Size([1000, 1, 1])


In [14]:
print(x.shape)
print(x)

(1000, 14)
[[0.6080777  0.866517   0.8081265  ... 0.10111862 0.7931133  0.6441473 ]
 [0.15858662 0.51983744 0.7864651  ... 0.08611625 0.8781233  0.38189042]
 [0.03201604 0.7092354  0.71404564 ... 0.38001066 0.5189433  0.41213477]
 ...
 [0.9158994  0.7052829  0.83027744 ... 0.10236162 0.55097026 0.4493503 ]
 [0.10370034 0.7762338  0.04077452 ... 0.63663274 0.9343347  0.7985732 ]
 [0.10690528 0.24554747 0.44383055 ... 0.5877984  0.3843271  0.342385  ]]


In [15]:



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(14,20 ,bias=True)
        self.layer2 = nn.Linear(20, 10,bias=True)
        self.layer3 = nn.Linear(10, 1,bias=True)


    def forward(self, x):
        # Max pooling over a (2, 2) window
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = torch.sigmoid(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = torch.sigmoid(self.layer3(x))
        #x = F.relu(self.layer4(x))
        #x = F.relu(self.layer5(x))
        #x = torch.sigmoid(self.layer6(x))
        return x


net = Net()
print(net)

Net(
  (layer1): Linear(in_features=14, out_features=20, bias=True)
  (layer2): Linear(in_features=20, out_features=10, bias=True)
  (layer3): Linear(in_features=10, out_features=1, bias=True)
)


In [16]:
params = list(net.parameters())
print(len(params))
print(params[2].size())

6
torch.Size([10, 20])


In [18]:
x_data=torch.tensor(x)
input = x_data
out = net(input)
print(out)
print(out.shape)

tensor([[0.5705],
        [0.5709],
        [0.5721],
        [0.5721],
        [0.5676],
        [0.5665],
        [0.5691],
        [0.5685],
        [0.5673],
        [0.5699],
        [0.5695],
        [0.5723],
        [0.5691],
        [0.5703],
        [0.5715],
        [0.5675],
        [0.5683],
        [0.5714],
        [0.5733],
        [0.5698],
        [0.5675],
        [0.5664],
        [0.5682],
        [0.5689],
        [0.5713],
        [0.5724],
        [0.5660],
        [0.5717],
        [0.5707],
        [0.5683],
        [0.5703],
        [0.5667],
        [0.5691],
        [0.5683],
        [0.5690],
        [0.5715],
        [0.5704],
        [0.5700],
        [0.5698],
        [0.5690],
        [0.5684],
        [0.5697],
        [0.5670],
        [0.5699],
        [0.5719],
        [0.5692],
        [0.5712],
        [0.5690],
        [0.5707],
        [0.5702],
        [0.5708],
        [0.5701],
        [0.5704],
        [0.5684],
        [0.5707],
        [0

In [19]:
net.zero_grad()
out.backward(torch.randn(1000,1))

In [20]:
target=y3
target_new=target.view(1000,1)
print(target_new.shape)
criterion=nn.MSELoss()
loss=criterion(out,target_new)
print(loss)

torch.Size([1000, 1])
tensor(0.7655, grad_fn=<MseLossBackward>)


In [22]:
import torch.optim as optim
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-09, weight_decay=0.1, amsgrad=False)  #adam optimizer
    
for i in range(200):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output, target_new)
    loss.backward()
    optimizer.step()
    print(loss)


tensor(0.4447, grad_fn=<MseLossBackward>)
tensor(0.4456, grad_fn=<MseLossBackward>)
tensor(0.4449, grad_fn=<MseLossBackward>)
tensor(0.4442, grad_fn=<MseLossBackward>)
tensor(0.4441, grad_fn=<MseLossBackward>)
tensor(0.4443, grad_fn=<MseLossBackward>)
tensor(0.4446, grad_fn=<MseLossBackward>)
tensor(0.4449, grad_fn=<MseLossBackward>)
tensor(0.4452, grad_fn=<MseLossBackward>)
tensor(0.4452, grad_fn=<MseLossBackward>)
tensor(0.4451, grad_fn=<MseLossBackward>)
tensor(0.4449, grad_fn=<MseLossBackward>)
tensor(0.4447, grad_fn=<MseLossBackward>)
tensor(0.4445, grad_fn=<MseLossBackward>)
tensor(0.4444, grad_fn=<MseLossBackward>)
tensor(0.4444, grad_fn=<MseLossBackward>)
tensor(0.4445, grad_fn=<MseLossBackward>)
tensor(0.4447, grad_fn=<MseLossBackward>)
tensor(0.4448, grad_fn=<MseLossBackward>)
tensor(0.4450, grad_fn=<MseLossBackward>)
tensor(0.4450, grad_fn=<MseLossBackward>)
tensor(0.4450, grad_fn=<MseLossBackward>)
tensor(0.4449, grad_fn=<MseLossBackward>)
tensor(0.4448, grad_fn=<MseLossBac

In [ ]:

class Net2(nn.Module):

    def __init__(self):
        super(Net2, self).__init__()

        self.layer1 = nn.Linear(14, 120)
        self.layer2 = nn.Linear(120, 120)
        self.layer3 = nn.Linear(120, 120)
        self.layer4 = nn.Linear(120, 84)
        self.layer5 = nn.Linear(84, 42)
        self.layer6 = nn.Linear(42, 7)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.layer1(x))
        x = x+F.relu(self.layer2(x))
        x = x+F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = F.tanh(self.layer5(x))
        x = F.relu(self.layer6(x))
        return x


net2 = Net2()
print(net2)

In [ ]:
params2 = list(net2.parameters())
print(len(params2))
print(params2[0].size())

In [ ]:
x_data=torch.tensor(x)
input = x_data
out = net2(input)
print(out)
print(out.shape)

In [ ]:
target=y1[0:1000,:,:]
target_new=target.view(1000,7)

In [ ]:
import torch.optim as optim
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-06, weight_decay=0.1, amsgrad=False)  #adam optimizer
    
for i in range(200):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net2(input)
    loss = criterion(output, target_new)
    loss.backward()
    optimizer.step()
    print(loss)
